<a href="https://colab.research.google.com/github/maham-patel/CI--EA-implementation/blob/main/EA%20class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class EA_imp:
  
  def __init__(self, genes, population_size, offspring_size, generations, mutation_rate, iterations, parent_ss, child_ss):
    self.population_size = population_size
    self.offspring_size = offspring_size
    self.generations = generations
    self.mutation_rate = mutation_rate
    self.iterations = iterations
    self.BSF_generation = []
    self.ASF_generation = []
    self.BSF_iter = []
    self.ASF_iter = []
    self.cur_generation = 1
    self.population = []

    



BLA
